### Input

In [ ]:
import json, dbm

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
with dbm.open('cache', 'c') as db:
    map_coordinates = json.loads(db[b'map_coordinates'])

In [ ]:
with dbm.open('cache', 'c') as db:
    mapping = json.loads(db[b'mapping'])

In [ ]:
DF = pd.read_csv("flights.csv")
DF = DF.loc[:, ["departure", "arrival"]]
DF["order"] = range(len(DF))

In [ ]:
for i in range(len(DF)):
    if DF.departure.iloc[i] in mapping:
        DF.departure.iloc[i] = mapping[DF.departure.iloc[i]]
    if DF.arrival.iloc[i] in mapping:
        DF.arrival.iloc[i] = mapping[DF.arrival.iloc[i]]

### To DF

In [ ]:
DF_c = pd.DataFrame(map_coordinates).T
DF_c.columns = ["latitude", "longitude"]

In [ ]:
DF = DF.merge(DF_c, left_on="departure", right_index=True)
DF = DF.rename({"latitude":"latitude_dep", "longitude":"longitude_dep"}, axis=1)
DF

In [ ]:
DF = DF.merge(DF_c, left_on="arrival", right_index=True)
DF = DF.rename({"latitude":"latitude_arr", "longitude":"longitude_arr"}, axis=1)
DF

In [ ]:
DF = DF.sort_values("order")

### Plot trip

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [ ]:
llon=-179
ulon=180
llat=-70
ulat=80

my_map=Basemap(projection='merc',
            resolution = 'l', area_thresh = 1000.0,
            llcrnrlon=llon, llcrnrlat=llat,urcrnrlon=ulon, urcrnrlat=ulat)


In [ ]:
def plot_trip(trip, df_base):
    my_map.drawlsmask(land_color='grey', ocean_color='skyblue')
    xs,ys = my_map(np.asarray(df_base.longitude), np.asarray(df_base.latitude))
    df_base['x']= xs.tolist()
    df_base['y'] =ys.tolist()

    for index,row in df_base.iterrows():
        my_map.plot(row.x,row.y,markerfacecolor='black',
                   markeredgecolor='white',marker='o',markersize=5)
    
    xs,ys = my_map(trip.loc[["longitude_dep", "longitude_arr"]].values, 
                   trip.loc[["latitude_dep", "latitude_arr"]].values,)
    
    my_map.plot(xs.tolist(), ys.tolist(), markerfacecolor='red',
                   markeredgecolor='white',marker='o',
               c="r")
    

In [ ]:
trip = DF.loc[123]
trip

In [ ]:
plot_trip(trip, DF_c)
plt.show()

### Animate

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
from celluloid import Camera # getting the camera
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML # to show the animation in Jupyter
fig, ax = plt.subplots() # creating my fig
camera = Camera(fig)# the camera gets the fig we'll plot

for i, trip in tqdm_notebook(DF.iterrows(), total=len(DF)):
    title = ax.text(0.5,0.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                transform=ax.transAxes, ha="center")
    plot_trip(trip, DF_c)
    title.set_text(f"{trip.departure} -> {trip.arrival}")
    camera.snap() # the camera takes a snapshot of the plot
    plt.savefig(f"images/img_{i}.png")

animation = camera.animate() # animation ready
HTML(animation.to_jshtml()) # displaying the animation


### Save to file

In [ ]:
from copy import deepcopy

In [ ]:
my_map0=Basemap(projection='merc',
            resolution = 'l', area_thresh = 1000.0,
            llcrnrlon=llon, llcrnrlat=llat,urcrnrlon=ulon, urcrnrlat=ulat)
    
for i, trip in tqdm_notebook(DF.iterrows(), total=len(DF)):
    if i < 110:
        continue
    my_map = deepcopy(my_map0)
    
    fig, ax = plt.subplots() # creating my fig
    
    title = ax.text(0.5,0.85, "", bbox={'facecolor':'w', 'alpha':0.5, 'pad':5},
                transform=ax.transAxes, ha="center")
    plot_trip(trip, DF_c)
    title.set_text(f"{trip.departure} -> {trip.arrival}")
    plt.savefig(f"images/img_{i}.png")
